In [1]:
# from langchain_community.document_loaders import WebBaseLoader
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain_chroma import Chroma
# from langchain_google_genai import GoogleGenerativeAIEmbeddings
# from langchain_google_genai import ChatGoogleGenerativeAI
# from langchain_community.document_loaders import PyPDFLoader
# from pydantic import BaseModel, Field
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.runnables import RunnablePassthrough
# from langchain_core.runnables import chain
# from langchain.chains import create_history_aware_retriever
# from langchain_core.prompts import MessagesPlaceholder
# from langchain_core.messages import AIMessage, HumanMessage
# from IPython.display import Markdown, display


# import os

# api_key = os.getenv("GOOGLE_API_KEY")

# embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=api_key)
# llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash",temperature=0.3, max_tokens=1000, api_key=api_key )
# chat_history = []


# # chroma db persist directory
# persist_directory = "local_chroma_db"

# # general information
# vectorstore_general = Chroma(persist_directory=persist_directory, embedding_function=embeddings, collection_name="general")
# retriever_general = vectorstore_general.as_retriever(search_kwargs={"k": 10})


#  # sajith's manifesto
# vectorstore_sajith = Chroma(persist_directory=persist_directory, embedding_function=embeddings, collection_name="sajith_premadasa")
# retriever_sajith = vectorstore_sajith.as_retriever(search_kwargs={"k": 10})

# # akd's manifesto
# vectorstore_akd = Chroma(persist_directory=persist_directory, embedding_function=embeddings, collection_name="anura_kumara_dissanayake")
# retriever_akd = vectorstore_akd.as_retriever(search_kwargs={"k": 10})

# # ranil's manifesto
# vectorstore_ranil = Chroma(persist_directory=persist_directory, embedding_function=embeddings, collection_name="ranil_wickramasinghe")
# retriever_ranil = vectorstore_ranil.as_retriever(search_kwargs={"k": 10})

# contextualize_q_system_prompt = (
#     "Given a chat history and the latest user question "
#     "which might reference context in the chat history, "
#     "formulate a standalone question which can be understood "
#     "without the chat history. Do NOT answer the question, "
#     "just reformulate it if needed and otherwise return it as is."
# )

# contextualize_q_prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", contextualize_q_system_prompt),
#         MessagesPlaceholder("chat_history"),
#         ("human", "{input}"),
#     ]
# )

# history_aware_retriever_general = create_history_aware_retriever(
#     llm, retriever_general, contextualize_q_prompt
# )

# history_aware_retriever_sajith = create_history_aware_retriever(
#     llm, retriever_sajith, contextualize_q_prompt
# )

# history_aware_retriever_akd = create_history_aware_retriever(
#     llm, retriever_akd, contextualize_q_prompt
# )

# history_aware_retriever_ranil = create_history_aware_retriever(
#     llm, retriever_ranil, contextualize_q_prompt
# )

# retrievers = {
#     "sajith_premadasa": history_aware_retriever_sajith,
#     "anura_kumara_dissanayake": history_aware_retriever_akd,
#     "ranil_wickramasinghe": history_aware_retriever_ranil,
# }

# class SearchAndCompare(BaseModel):
#     """Search for information about a person or compare informations about persons."""
#     queryType: str = Field(
#         ...,
#         description="Query type. Should be `search` or `compare` or `general`. if there's only one person name it's search, if there are many person's name it's compare, or it can be a general question which does not require any specific person",)

#     query: str = Field(
#         ...,
#         description="Query to look up or query to compare",
#     )

#     candidates: int = Field(
#         ...,
#         description="Number of persons to search or compare. can be 0 for general questions",
#     )

#     person1: str = Field(
#         ...,
#         description="Person to look things up for or persons to compare. Should be `sajith_premadasa` or `anura_kumara_dissanayake` or `ranil_wickramasinghe` or can be 'null'.",
#     )
#     person2: str = Field(
#         ...,
#         description="Person to look things up for or persons to compare. Should be `sajith_premadasa` or `anura_kumara_dissanayake` or `ranil_wickramasinghe` or can be 'null'.",
#     )
#     person3: str = Field(
#         ...,
#         description="Person to look things up for or persons to compare. Should be `sajith_premadasa` or `anura_kumara_dissanayake` or `ranil_wickramasinghe` or can be 'null'.",
#     )

# structured_llm_query = llm.with_structured_output(SearchAndCompare)

# @chain
# def query_analyzer_chain(question):
#     system_query = """You have the ability to determine whether the user question is general, or it is related to a specific person or it is a comparison between multiple persons. if you can't find the type set it as general"""
#     prompt_query = ChatPromptTemplate.from_messages(
#         [
#             ("system", system_query),
#             ("human", "{question}"),
#         ]
#     )
#     prompt_query = prompt_query.format_messages(question=question)
#     response = structured_llm_query.invoke(prompt_query)

#     return response


# @chain
# def qa_chain(question):
#     response = query_analyzer_chain.invoke(question)
#     # print(response)
#     if response.queryType == "search" or response.queryType == "compare":
#         if response.queryType == "search":
#             retriever = retrievers[response.person1]
#             retrieved_docs = retriever.invoke({"input":response.query, "chat_history": chat_history})

#             prompt = (
#             "system :"
#             "You are an assistant for question-answering tasks. "
#             "Use the following pieces of retrieved context to answer "
#             "the question. If you don't know the answer, say that you "
#             "don't know."
#             "\n\n"
#             "{context}"
#             "\n\n"

#             "chat_history :" 
#             "{chat_history}"

#             "human :"
#             "{question}"
#             ).format(context=retrieved_docs, question=question, chat_history=chat_history)

#             result = llm.invoke(prompt)

#             return result
    
#         elif response.queryType == "compare":
#             retriever1 = retrievers[response.person1]
#             retrieved_docs1 = retriever1.invoke({"input":response.query, "chat_history": chat_history})

#             if response.person2 != 'null':
#                 retriever2 = retrievers[response.person2]
#                 retrieved_docs2 = retriever2.invoke({"input":response.query, "chat_history": chat_history})
#             else:
#                 retrieved_docs2 = ''

#             if response.person3 != 'null':
#                 retriever3 = retrievers[response.person3]
#                 retrieved_docs3 = retriever3.invoke({"input":response.query, "chat_history": chat_history})
#             else:
#                 retrieved_docs3 = ''

#             prompt = (
#             "system :"
#             "You are an assistant for comparing manifestos. "
#             "Use the following pieces of retrieved context from different manifestos to answer "
#             "the question. If you don't know the answer, say that you "
#             "don't know."
#             "\n\n"
#             "{context1}"
#             "\n\n"
#             "{context2}"
#             "\n\n"
#             "{context3}"
#             "\n\n"

#             "chat_history :" 
#             "{chat_history}"

#             "human :"
#             "{question}"
#             ).format(context1=retrieved_docs1, context2=retrieved_docs2, context3=retrieved_docs3, question=question, chat_history=chat_history)

#             result = llm.invoke(prompt)

#             return result
#     else:
#         retriever = history_aware_retriever_general
#         retrieved_docs = retriever.invoke({"input":response.query, "chat_history": chat_history})

#         prompt = (
#             "system :"
#             "You are an assistant for question-answering tasks related to srilankan election."
#             "Use the following pieces of retrieved context to answer "
#             "the question. If you don't know the answer, say that you "
#             "don't know."
#             "or if the question is not much related to srilankan election say that this question is not related to srilankan election ass a election chatbot i can't provide you with answer this."
#             "\n\n"
#             "{context}"
#             "\n\n"

#             "chat_history :" 
#             "{chat_history}"

#             "human :"
#             "{question}"
#             ).format(context=retrieved_docs, question=question, chat_history=chat_history)

#         result = llm.invoke(prompt)
#         return result

# def chatbot(question):
#     result = qa_chain.invoke(question)
    
#     # retains only last 3 conversations in history
#     if len(chat_history) == 6:
#         chat_history.pop(0)
#         chat_history.pop(0)
    
#     chat_history.extend([
#             HumanMessage(content=question),
#             AIMessage(content=result.content),
#         ])
    
#     return result.content

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [15]:
# from langchain_community.document_loaders import WebBaseLoader
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain_chroma import Chroma
# from langchain_google_genai import GoogleGenerativeAIEmbeddings
# from langchain_google_genai import ChatGoogleGenerativeAI
# from langchain_community.document_loaders import PyPDFLoader
# from pydantic import BaseModel, Field
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.runnables import RunnablePassthrough
# from langchain_core.runnables import chain
# from langchain.chains import create_history_aware_retriever
# from langchain_core.prompts import MessagesPlaceholder
# from langchain_core.messages import AIMessage, HumanMessage
# from IPython.display import Markdown, display


# import os

# api_key = os.getenv("GOOGLE_API_KEY")

# embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=api_key)
# llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash",temperature=0.3, max_tokens=1000, api_key=api_key )
# chat_history = []


# # chroma db persist directory
# persist_directory = "local_chroma_db"

# # general information
# vectorstore_general = Chroma(persist_directory=persist_directory, embedding_function=embeddings, collection_name="general")
# retriever_general = vectorstore_general.as_retriever(search_kwargs={"k": 10})


#  # sajith's manifesto
# vectorstore_sajith = Chroma(persist_directory=persist_directory, embedding_function=embeddings, collection_name="sajith_premadasa")
# retriever_sajith = vectorstore_sajith.as_retriever(search_kwargs={"k": 10})

# # akd's manifesto
# vectorstore_akd = Chroma(persist_directory=persist_directory, embedding_function=embeddings, collection_name="anura_kumara_dissanayake")
# retriever_akd = vectorstore_akd.as_retriever(search_kwargs={"k": 10})

# # ranil's manifesto
# vectorstore_ranil = Chroma(persist_directory=persist_directory, embedding_function=embeddings, collection_name="ranil_wickramasinghe")
# retriever_ranil = vectorstore_ranil.as_retriever(search_kwargs={"k": 10})

# contextualize_q_system_prompt = (
#     "Given a chat history and the latest user question "
#     "which might reference context in the chat history, "
#     "formulate a standalone question which can be understood "
#     "without the chat history. Do NOT answer the question, "
#     "just reformulate it if needed and otherwise return it as is."
# )

# contextualize_q_prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", contextualize_q_system_prompt),
#         MessagesPlaceholder("chat_history"),
#         ("human", "{input}"),
#     ]
# )

# history_aware_retriever_general = create_history_aware_retriever(
#     llm, retriever_general, contextualize_q_prompt
# )

# history_aware_retriever_sajith = create_history_aware_retriever(
#     llm, retriever_sajith, contextualize_q_prompt
# )

# history_aware_retriever_akd = create_history_aware_retriever(
#     llm, retriever_akd, contextualize_q_prompt
# )

# history_aware_retriever_ranil = create_history_aware_retriever(
#     llm, retriever_ranil, contextualize_q_prompt
# )

# retrievers = {
#     "sajith_premadasa": retriever_sajith,
#     "anura_kumara_dissanayake": retriever_akd,
#     "ranil_wickramasinghe": retriever_ranil,
# }

# class SearchAndCompare(BaseModel):
#     """Search for information about a person or compare informations about persons."""
#     queryType: str = Field(
#         ...,
#         description="Query type. Should be `search` or `compare` or `general`. if there's only one person name it's search, if there are many person's name it's compare, or it can be a general question which does not require any specific person",)

#     query: str = Field(
#         ...,
#         description="Query to look up or query to compare",
#     )

#     candidates: int = Field(
#         ...,
#         description="Number of persons to search or compare. can be 0 for general questions",
#     )

#     person1: str = Field(
#         ...,
#         description="Person to look things up for or persons to compare. Should be `sajith_premadasa` or `anura_kumara_dissanayake` or `ranil_wickramasinghe` or can be 'null'.",
#     )
#     person2: str = Field(
#         ...,
#         description="Person to look things up for or persons to compare. Should be `sajith_premadasa` or `anura_kumara_dissanayake` or `ranil_wickramasinghe` or can be 'null'.",
#     )
#     person3: str = Field(
#         ...,
#         description="Person to look things up for or persons to compare. Should be `sajith_premadasa` or `anura_kumara_dissanayake` or `ranil_wickramasinghe` or can be 'null'.",
#     )

# structured_llm_query = llm.with_structured_output(SearchAndCompare)

# @chain
# def query_analyzer_chain(question):
#     system_query = """You have the ability to determine whether the user question is general, or it is related to a specific person or it is a comparison between multiple persons. if you can't find the type set it as general"""
#     prompt_query = ChatPromptTemplate.from_messages(
#         [
#             ("system", system_query),
#             ("human", "{question}"),
#         ]
#     )
#     prompt_query = prompt_query.format_messages(question=question)
#     response = structured_llm_query.invoke(prompt_query)

#     return response


# @chain
# def qa_chain(question):
#     response = query_analyzer_chain.invoke(question)
#     # print(response)
#     if response.queryType == "search" or response.queryType == "compare":
#         if response.queryType == "search":
#             retriever = retrievers[response.person1]
#             retrieved_docs = retriever.invoke(response.query)

#             prompt = (
#             "system :"
#             "You are an assistant for question-answering tasks. "
#             "Use the following pieces of retrieved context to answer "
#             "the question. If you don't know the answer, say that you "
#             "don't know."
#             "\n\n"
#             "{context}"
#             "\n\n"

#             "chat_history :" 
#             "{chat_history}"

#             "human :"
#             "{question}"
#             ).format(context=retrieved_docs, question=question, chat_history=chat_history)

#             result = llm.invoke(prompt)

#             return result
    
#         elif response.queryType == "compare":
#             retriever1 = retrievers[response.person1]
#             retrieved_docs1 = retriever1.invoke(response.query)

#             if response.person2 != 'null':
#                 retriever2 = retrievers[response.person2]
#                 retrieved_docs2 = retriever2.invoke(response.query)
#             else:
#                 retrieved_docs2 = ''

#             if response.person3 != 'null':
#                 retriever3 = retrievers[response.person3]
#                 retrieved_docs3 = retriever3.invoke(response.query)
#             else:
#                 retrieved_docs3 = ''

#             prompt = (
#             "system :"
#             "You are an assistant for comparing manifestos. "
#             "Use the following pieces of retrieved context from different manifestos to answer "
#             "the question. If you don't know the answer, say that you "
#             "don't know."
#             "\n\n"
#             "{context1}"
#             "\n\n"
#             "{context2}"
#             "\n\n"
#             "{context3}"
#             "\n\n"

#             "chat_history :" 
#             "{chat_history}"

#             "human :"
#             "{question}"
#             ).format(context1=retrieved_docs1, context2=retrieved_docs2, context3=retrieved_docs3, question=question, chat_history=chat_history)

#             result = llm.invoke(prompt)

#             return result
#     else:
#         retriever = retriever_general
#         retrieved_docs = retriever.invoke(response.query)

#         prompt = (
#             "system :"
#             "You are an assistant for question-answering tasks related to srilankan election."
#             "Use the following pieces of retrieved context to answer "
#             "the question. If you don't know the answer, say that you "
#             "don't know."
#             "or if the question is not much related to srilankan election say that this question is not related to srilankan election ass a election chatbot i can't provide you with answer this."
#             "\n\n"
#             "{context}"
#             "\n\n"

#             "chat_history :" 
#             "{chat_history}"

#             "human :"
#             "{question}"
#             ).format(context=retrieved_docs, question=question, chat_history=chat_history)

#         result = llm.invoke(prompt)
#         return result

# def chatbot(question):
#     result = qa_chain.invoke(question)
    
#     # retains only last 3 conversations in history
#     if len(chat_history) == 6:
#         chat_history.pop(0)
#         chat_history.pop(0)
    
#     chat_history.extend([
#             HumanMessage(content=question),
#             AIMessage(content=result.content),
#         ])
    
#     return result.content

In [6]:
# import chromadb
# from chromadb.config import Settings
# class Chat_db:
#     def __init__(self, persist_directory="local_chroma_db"):
#         self.persist_directory = persist_directory             
#         self.client_settings = Settings(is_persistent= True, persist_directory= persist_directory, anonymized_telemetry=False)
#         self.persistent_client = chromadb.Client(settings= self.client_settings)
#         self.doc_collection = self.persistent_client.get_or_create_collection(name = "general")
       
#         #I thought I could just use self.persistent_client for querying but it did not work. 
#         # had to create a PersistentClient to use for querying.
#         self.queryclient = chromadb.PersistentClient(path= persist_directory, settings= self.client_settings)
        

#     def chat_over_documents(self, collection_name, query, k = 5):
#         collection = self.queryclient.get_collection(name=collection_name)
#         results = collection.query(query_texts= query, n_results= k)
#         flat_results = [item for sublist in results['documents'] for item in sublist] 
#         return flat_results

In [8]:
# document_handler = Chat_db()

# response = document_handler.chat_over_documents(query="ranil", collection_name="general",k=5)

# response

In [19]:
question = "what sajith and anura and ranil said about education"
display(Markdown(chatbot(question)))

Here's a comparison of what Sajith, Anura, and Ranil said about education in their manifestos:

**Sajith Premadasa:**

* **Focus on quality and accessibility:**  He emphasizes making education more accessible and improving its quality to build a competitive workforce.
* **Smart Schools:**  He aims to transform every school into a smart school with 100% electricity, water facilities, and physical resources.
* **Digital Learning:**  He plans to expand the "Sakwala" program to provide access to digital learning platforms in schools, technical and vocational institutions, and universities.
* **Teacher Support:**  He promises free public transportation for all teachers and plans to address salary, pension, and service issues in the education sector.
* **Early Childhood Education:**  He recognizes the importance of ECE and plans to establish a regulatory authority with a multi-stakeholder advisory committee.
* **"STEEAM" Education:**  He wants to prioritize Science, Technology, English, Engineering, Arts, and Mathematics in all institutions by January 2025.
* **English Proficiency:**  He aims to design primary and secondary education with modern technology to enable students to understand, speak, and write English from an early stage.
* **Vocational Training:**  He plans to modernize and standardize vocational training curricula to align with market demands.

**Anura Dissanayake:**

* **Engineering Education Reform:**  He focuses on reforming and restructuring engineering education to produce industry-related practitioners.
* **Equal Education Opportunities:**  He aims to provide equal education opportunities around the country with free access to education materials.
* **Promoting IoT Applications:**  He wants to promote IoT applications for the development of IoT networks (e.g., smart metering).
* **Terrestrial TV for Education:**  He identifies terrestrial TV as an efficient medium for educational services and plans to improve technical infrastructure.

**Ranil Wickremesinghe:**

* **Modernizing School Infrastructure:**  He acknowledges the need to modernize school infrastructure to meet the demands of the technological era.
* **Teacher Training:**  He plans to equip teachers with new knowledge and training in 2025.
* **School Boards:**  He aims to complete the establishment of School Boards with representatives from School Development Societies and Past Pupils Associations by 2027.
* **Soft Skills and Moral Values:**  He wants to revise school education with a focus on soft skills, life skills, and moral values.
* **"English for All" Program:**  He plans to implement an "English for All" program to provide English education to all schoolchildren within the next decade.
* **Vocational Skills Sri Lanka (VSSL):**  He proposes establishing a new entity called VSSL in 2025 to amalgamate the National Apprentice and Industrial Training Authority and the Vocational Training Authority of Sri Lanka.
* **Provincial Vocational Education Boards:**  He aims to establish Provincial Vocational Education Boards in all nine provinces by the end of 2026.
* **Technical and Vocational and Management University Colleges:**  He plans to open these colleges in all provinces with faculties in all districts.

**Key Differences:**

* **Sajith** focuses on making education more accessible and improving its quality, with a strong emphasis on digital learning and teacher support.
* **Anura** focuses on reforming engineering education and promoting technology in education.
* **Ranil** focuses on modernizing school infrastructure, improving teacher training, and promoting soft skills and moral values.

**Similarities:**

* All three candidates recognize the importance of education and have plans to improve it in various ways.
* They all emphasize the need for a skilled workforce and the importance of technology in education. 
